## We remove all queries from the part starts with any of these:

&format

&timeout

&debug

&run

In [152]:
import pandas as pd
import re

def remove_keywords(query):
    # List of keywords to remove
    keywords = ["&format", "&timeout", "&debug", "&run"]

    # Regular expression to find any of the keywords in the query
    regex = '|'.join(re.escape(keyword) for keyword in keywords)

    # Remove the matched part of the query and everything after it
    query = re.sub(rf'({regex}).*', '', query)

    return query.strip()

def process_csv(input_file, output_file):
    # Read the CSV file into a Pandas DataFrame
    df = pd.read_csv(input_file)

    # Process the "query" column using the remove_keywords function
    df['query'] = df['query'].apply(remove_keywords)

    # Save the updated DataFrame to a new CSV file
    df.to_csv(output_file, index=False)

if __name__ == "__main__":
    input_csv_file = "bio2rdf_sparql_logs_processed_01-2019_to_07-2021.csv"
    output_csv_file = "removedformats_bio2rdf_logs.csv"
    process_csv(input_csv_file, output_csv_file)


/tmp/ipykernel_5259/2559306569.py:18: DtypeWarning: Columns (1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_file)


## We extract unique queries

In [153]:
import pandas as pd

def extract_unique_queries(input_file, output_file):
    try:
        # Read the CSV file into a pandas DataFrame
        df = pd.read_csv(input_file)

        # Extract unique values from the 'query' column
        unique_queries = df['query'].unique()

        # Create a new DataFrame with the unique queries
        unique_queries_df = pd.DataFrame({'query': unique_queries})

        # Write the unique queries DataFrame to a new CSV file
        unique_queries_df.to_csv(output_file, index=False)

        print("Unique queries extracted and written to", output_file)

    except FileNotFoundError:
        print("Error: Input CSV file not found.")
    except Exception as e:
        print("An error occurred:", e)

# Replace 'input_file.csv' with the name of your CSV file and 'output_file.csv' with the desired output filename
extract_unique_queries('removedformats_bio2rdf_logs.csv', 'unique_bio2rdf_sparql_logs.csv')


/tmp/ipykernel_5259/1392910086.py:6: DtypeWarning: Columns (1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_file)


Unique queries extracted and written to unique_bio2rdf_sparql_logs.csv


## we add following list to all queries:

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

PREFIX owl: <http://www.w3.org/2002/07/owl#>

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

PREFIX foaf: <http://xmlns.com/foaf/0.1/>

PREFIX dc: <http://purl.org/dc/elements/1.1/>

PREFIX dcterms: <http://purl.org/dc/terms/>

PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

PREFIX schema: <http://schema.org/>

PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>


In [ ]:
import pandas as pd

def add_prefix(query):
    # The common prefix to add to each query
    prefix = """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX schema: <http://schema.org/>
PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>

"""
    return prefix + query

def add_prefix_to_csv(input_file, output_file):
    # Read the CSV file into a Pandas DataFrame
    df = pd.read_csv(input_file)

    # Add the prefix to the "query" column using the add_prefix function
    df['query'] = df['query'].apply(add_prefix)

    # Save the updated DataFrame to a new CSV file
    df.to_csv(output_file, index=False)

if __name__ == "__main__":
    input_csv_file = "unique_bio2rdf_sparql_logs.csv"  # Replace with the path to your input CSV file
    output_csv_file = "prefix_unique_bio2rdf_sparql_logs.csv"  # Replace with the path to your output CSV file
    add_prefix_to_csv(input_csv_file, output_csv_file)

## We count subjects to calculate complexity

In [157]:
import pandas as pd

def count_subject_complexity(parsed_query):
    return parsed_query.count("subject")

# Replace 'input.csv' with the name of your input CSV file and 'output.csv' with the desired output CSV file name
input_file = 'translated_queries.csv'
output_file = 'complexity.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(input_file)

# Create a new column 'complexity' by counting the occurrences of 'subject' in 'parsed_query'
df['complexity'] = df['Parsed_Query'].apply(count_subject_complexity)

# Create a new DataFrame with 'query' and 'complexity' columns
result_df = df[['Query','Parsed_Query', 'complexity']]

# Save the result DataFrame to a new CSV file
result_df.to_csv(output_file, index=False)


In [111]:
import json

def count_variable_and_namednode_terms(json_text):
    parsed_json = json.loads(json_text)
    where_clause = parsed_json.get('where', [])

    variable_count = 0
    namednode_count = 0

    for clause in where_clause:
        if 'triples' in clause:
            for triple in clause['triples']:
                if 'subject' in triple and 'termType' in triple['subject'] and triple['subject']['termType'] == 'Variable':
                    variable_count += 1
                if 'predicate' in triple and 'termType' in triple['predicate'] and triple['predicate']['termType'] == 'NamedNode':
                    namednode_count += 1
                if 'object' in triple and 'termType' in triple['object'] and triple['object']['termType'] == 'Variable':
                    variable_count += 1
                if 'object' in triple and 'termType' in triple['object'] and triple['object']['termType'] == 'NamedNode':
                    namednode_count += 1

    return variable_count, namednode_count

if __name__ == "__main__":
    text = '''
    {
      "queryType": "SELECT",
      "variables": [
        {
          "termType": "Variable",
          "value": "p"
        },
        {
          "termType": "Variable",
          "value": "c"
        }
      ],
      "where": [
        {
          "type": "bgp",
          "triples": [
            {
              "subject": {
                "termType": "Variable",
                "value": "p"
              },
              "predicate": {
                "termType": "NamedNode",
                "value": "http://www.w3.org/1999/02/22-rdf-syntax-ns#type"
              },
              "object": {
                "termType": "NamedNode",
                "value": "http://dbpedia.org/ontology/Artist"
              }
            },
            {
              "subject": {
                "termType": "Variable",
                "value": "p"
              },
              "predicate": {
                "termType": "NamedNode",
                "value": "http://dbpedia.org/ontology/birthPlace"
              },
              "object": {
                "termType": "Variable",
                "value": "c"
              }
            },
            {
              "subject": {
                "termType": "Variable",
                "value": "c"
              },
              "predicate": {
                "termType": "NamedNode",
                "value": "http://xmlns.com/foaf/0.1/name"
              },
              "object": {
                "termType": "Literal",
                "value": "York",
                "language": "en",
                "datatype": {
                  "termType": "NamedNode",
                  "value": "http://www.w3.org/1999/02/22-rdf-syntax-ns#langString"
                }
              }
            }
          ]
        }
      ],
      "type": "query",
      "prefixes": {
        "dbpedia-owl": "http://dbpedia.org/ontology/"
      }
    }
    '''

    variable_count, namednode_count = count_variable_and_namednode_terms(text)
    print("Number of 'Variable' terms after 'where':", variable_count)
    print("Number of 'NamedNode' terms:", namednode_count)


Number of 'Variable' terms after 'where': 4
Number of 'NamedNode' terms: 4


In [117]:
import json

text = '''
    {
      "queryType": "SELECT",
      "variables": "x y",
      "where": [
        {
          "type": "bgp",
          "triples": [
            {
              "subject": { "termType": "Variable", "value": "x" },
              "predicate": { "termType": "NamedNode", "value": "http://www.w3.org/1999/02/22-rdf-syntax-ns#type" },
              "object": { "termType": "Variable", "value": "y" }
            }
          ]
        },
        {
          "type": "group",
          "patterns": [
            {
              "queryType": "SELECT",
              "variables": [ { "termType": "Variable", "value": "y" } ],
              "where": [
                {
                  "type": "bgp",
                  "triples": [
                    {
                      "subject": { "termType": "Variable", "value": "y" },
                      "predicate": { "termType": "Variable", "value": "o" },
                      "object": { "termType": "Variable", "value": "d" }
                    }
                  ]
                }
              ],
              "type": "query"
            }
          ]
        }
      ],
      "type": "query",
      "prefixes": {}
    }
    '''

# Parse the JSON text into a Python dictionary
data = json.loads(text)

# Function to count variables and named nodes in the bgp section
def count_in_bgp(bgp_data):
    variable_count = 0
    named_node_count = 0

    if "triples" in bgp_data:
        triples = bgp_data["triples"]
        for triple in triples:
            if triple["subject"]["termType"] == "Variable":
                variable_count += 1
            elif triple["subject"]["termType"] == "NamedNode":
                named_node_count += 1
            if triple["predicate"]["termType"] == "Variable":
                variable_count += 1
            elif triple["predicate"]["termType"] == "NamedNode":
                named_node_count += 1
            if triple["object"]["termType"] == "Variable":
                variable_count += 1
            elif triple["object"]["termType"] == "NamedNode":
                named_node_count += 1

    return variable_count, named_node_count

# Initialize counts
total_variable_count = 0
total_named_node_count = 0

# Traverse through all "bgp" sections and count variables and named nodes in bgps
def traverse(data):
    global total_variable_count, total_named_node_count
    if isinstance(data, dict):
        if "type" in data and data["type"] == "bgp":
            variable_count, named_node_count = count_in_bgp(data)
            total_variable_count += variable_count
            total_named_node_count += named_node_count
        for key, value in data.items():
            traverse(value)
    elif isinstance(data, list):
        for item in data:
            traverse(item)

# Start traversing from the main JSON dictionary
traverse(data)

# Print the results
print("Total Variable Count:", total_variable_count)
print("Total Named Node Count:", total_named_node_count)


Total Variable Count: 5
Total Named Node Count: 1


## We calculate Informativness

In [123]:
 import json
import csv

# Function to count variables and named nodes in the bgp section
def count_in_bgp(bgp_data):
    variable_count = 0
    named_node_count = 0

    if "triples" in bgp_data:
        triples = bgp_data["triples"]
        for triple in triples:
            if triple["subject"]["termType"] == "Variable":
                variable_count += 1
            elif triple["subject"]["termType"] == "NamedNode":
                named_node_count += 1
            if triple["predicate"]["termType"] == "Variable":
                variable_count += 1
            elif triple["predicate"]["termType"] == "NamedNode":
                named_node_count += 1
            if triple["object"]["termType"] == "Variable":
                variable_count += 1
            elif triple["object"]["termType"] == "NamedNode":
                named_node_count += 1

    return variable_count, named_node_count

# Function to calculate informativeness and update CSV rows
def calculate_informativeness(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w', newline='') as outfile:
        reader = csv.DictReader(infile)
        fieldnames = reader.fieldnames + ['Informativeness']
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()

        for row in reader:
            parsed_query = row['Parsed_Query']

            try:
                data = json.loads(parsed_query)
            except json.JSONDecodeError:
                print(f"Error parsing JSON in row: {row}")
                continue

            total_variable_count = 0
            total_named_node_count = 0

            # Traverse through all "bgp" sections and count variables and named nodes in bgps
            def traverse(data):
                nonlocal total_variable_count, total_named_node_count
                if isinstance(data, dict):
                    if "type" in data and data["type"] == "bgp":
                        variable_count, named_node_count = count_in_bgp(data)
                        total_variable_count += variable_count
                        total_named_node_count += named_node_count
                    for key, value in data.items():
                        traverse(value)
                elif isinstance(data, list):
                    for item in data:
                        traverse(item)

            # Start traversing from the main JSON dictionary
            traverse(data)

            # Update the row with informativeness values
            informativeness = total_named_node_count / (total_variable_count + total_named_node_count)

            row['Informativeness'] = f'{informativeness}'
            writer.writerow(row)

# Provide the input and output file paths
input_csv_file = 'complexity.csv'
output_csv_file = 'informativness.csv'

# Call the function to calculate informativeness and update the output CSV file
calculate_informativeness(input_csv_file, output_csv_file)


Error parsing JSON in row: {'Query': 'select * where {[] a ?Concept} LIMIT 100&format=text/html&timeout=0&debug=on&run= Run Query', 'Parsed_Query': 'Error parsing the query.', 'complexity': '0'}
Error parsing JSON in row: {'Query': 'With the advantages of utilizing a job recruiter weighed towards the disadvantages you\'ll discover that most often using a job headhunter is not \nthe very best option for you or your organization. Whhen Mr.\nDelbart’s job att Orajge CH turned redundant, he was employed \nas Head of Support for Transport Publics Genevois, a personal \ncompany offering transport facilities to the public.\nLet me let you know slightly about Mr. Delbart’s career before analyzing \nthe common principles that it is best to observe.\nSince 2009, Betts Recruiting has been serving to veterans \nmake career dreams come true and serving to corporations scale \nfor growth. At the time, he was inquisitive about having a technical profession related to Information Technology (IT) impro

## Caculate similarity

In [158]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_similarity(queries):
    # Create a TF-IDF vectorizer to convert queries into numerical vectors
    vectorizer = TfidfVectorizer()

    # Convert the list of queries to TF-IDF vectors
    tfidf_matrix = vectorizer.fit_transform(queries)

    # Calculate cosine similarity between all pairs of queries
    similarity_matrix = cosine_similarity(tfidf_matrix)

    return similarity_matrix

def main():
    # Read the CSV file into a Pandas DataFrame
    input_csv_file = "informativness.csv"  # Replace with the path to your CSV file
    output_csv_file = "queries_with_similarity.csv"  # Replace with the desired output CSV file

    df = pd.read_csv(input_csv_file)

    # Get the queries from the DataFrame
    queries = df['Query'].tolist()

    # Calculate similarity between queries
    similarity_matrix = calculate_similarity(queries)

    # Add the similarity values as a new column for each query
    similarity_column = []
    for i in range(len(queries)):
        similarity_column.append(similarity_matrix[i].tolist())

    df['similarity'] = similarity_column

    # Save the DataFrame with the new similarity column to a new CSV file
    df.to_csv(output_csv_file, index=False)

if __name__ == "__main__":
    main()


## Calculate uniqueness of Q(i)

We define it as average of the Jaccard similarity(or cosine) scores between the query and all other queries.

U(Q(n)) = (Σ J(Q(n), Q(i))) / |Q|

## Jacard similarity and tokenizer

In [ ]:
import pandas as pd
# from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import pairwise_distances

# Step 1: Read the CSV file
csv_file_path = 'path_to_your_csv_file.csv'
df = pd.read_csv(csv_file_path)

# Step 2: Tokenize the queries
tokenized_queries = [query.split() for query in df['sparql_query_column']]

# Step 3: Convert queries into sets
query_sets = [set(tokens) for tokens in tokenized_queries]

# Step 4: Calculate Jaccard similarity
jaccard_similarities = 1 - pairwise_distances(query_sets, metric='jaccard')

# Step 5: Calculate the average uniqueness score for each query
average_uniqueness_scores = jaccard_similarities.mean(axis=1)

# Now 'average_uniqueness_scores' contains a single value representing the uniqueness of each query.


## Cosine similarity and vectores 

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import pairwise_distances

# Step 1: Read the CSV file
csv_file_path = 'path_to_your_csv_file.csv'
df = pd.read_csv(csv_file_path)

# Step 2: Convert queries into a list of strings
query_list = df['sparql_query_column'].tolist()

# Step 3: Calculate Cosine similarity
vectorizer = CountVectorizer()
query_vectors = vectorizer.fit_transform(query_list)
cosine_similarities = 1 - pairwise_distances(query_vectors, metric='cosine')

# Step 4: Calculate the average uniqueness score for each query
average_uniqueness_scores = cosine_similarities.mean(axis=1)

# Now 'average_uniqueness_scores' contains a single value representing the uniqueness of each query.


##  Optimized

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import vstack
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: Read the CSV file
csv_file_path = 'path_to_your_csv_file.csv'
df = pd.read_csv(csv_file_path)

# Step 2: Define the batch size for processing
batch_size = 1000

# Step 3: Initialize the vectorizer
vectorizer = TfidfVectorizer(use_idf=False)

# Step 4: Initialize the variable to store the final results
result_uniqueness_scores = []

# Step 5: Process data in batches
for i in range(0, len(df), batch_size):
    batch_queries = df['sparql_query_column'].iloc[i:i + batch_size].tolist()
    # Convert the queries into a sparse matrix
    batch_query_vectors = vectorizer.transform(batch_queries)
    # Calculate cosine similarity for the batch
    cosine_similarities = cosine_similarity(batch_query_vectors)
    # Compute the average uniqueness score for each query in the batch
    average_uniqueness_scores = cosine_similarities.mean(axis=1)
    result_uniqueness_scores.extend(average_uniqueness_scores)

# Now 'result_uniqueness_scores' contains a list of values representing the uniqueness of each query.


## Optimizing parsing of queries

we use streams to read and process data in chunks from the input CSV file. We then use the p-map library to parallelize the execution of SPARQL queries using multiple worker threads. This significantly speeds up the processing for large input files.

In [ ]:
const fs = require('fs');
const SparqlParser = require('sparqljs').Parser;
const csvParser = require('csv-parser');
const { unparse } = require('papaparse');
const { Worker, isMainThread, parentPort } = require('worker_threads');
const pMap = require('p-map');

// Create a new instance of the SPARQL parser
const parser = new SparqlParser();

// Function to execute a single SPARQL query and return the result
async function executeQuery(query) {
  try {
    const parsedQuery = parser.parse(query);
    // Here, you can execute the parsed query using your preferred SPARQL endpoint or RDF library
    // For the sake of example, we will just return the parsed query as a string
    return JSON.stringify(parsedQuery);
  } catch (error) {
    console.error('Error parsing query:', query);
    return 'Error parsing the query.';
  }
}

async function main() {
  // Read SPARQL queries from input CSV file and execute them in parallel
  const inputCsvFile = 'prefix_unique_bio2rdf_sparql_logs.csv';
  const outputCsvFile = 'translated_prefix_unique_bio2rdf_sparql_logs.csv';

  const csvData = [];

  const workerFunction = async (row) => {
    const sparqlQuery = row['query']; // Adjust the column name as per your input CSV format
    const parsedQuery = await executeQuery(sparqlQuery);
    return { Query: sparqlQuery, Parsed_Query: parsedQuery };
  };

  // Use pMap to parallelize the execution of queries
  const workerCount = 4; // Adjust the number of worker threads as per your CPU cores
  const workerResults = await pMap(
    fs.createReadStream(inputCsvFile).pipe(csvParser()),
    workerFunction,
    { concurrency: workerCount }
  );

  // Add the results to the csvData array
  csvData.push(...workerResults);

  // Write the results to output CSV file
  const csv = unparse(csvData, { header: true, delimiter: ',' });
  fs.writeFileSync(outputCsvFile, csv);

  console.log('All SPARQL queries executed and results written to output CSV file.');
}

if (isMainThread) {
  main();
} else {
  parentPort.once('message', (message) => {
    parentPort.postMessage(workerFunction(message));
  });
}


## Progress bar

In [ ]:
const fs = require('fs');
const SparqlParser = require('sparqljs').Parser;
const csvParser = require('csv-parser');
const { unparse } = require('papaparse');
const { Worker, isMainThread, parentPort } = require('worker_threads');
const pMap = require('p-map');

// Create a new instance of the SPARQL parser
const parser = new SparqlParser();

// Function to execute a single SPARQL query and return the result
async function executeQuery(query) {
  try {
    const parsedQuery = parser.parse(query);
    // Here, you can execute the parsed query using your preferred SPARQL endpoint or RDF library
    // For the sake of example, we will just return the parsed query as a string
    return JSON.stringify(parsedQuery);
  } catch (error) {
    console.error('Error parsing query:', query);
    return 'Error parsing the query.';
  }
}

async function main() {
  // Read SPARQL queries from input CSV file and execute them in parallel
  const inputCsvFile = 'prefix_unique_bio2rdf_sparql_logs.csv';
  const outputCsvFile = 'translated_prefix_unique_bio2rdf_sparql_logs.csv';

  const csvData = [];

  // Step 1: Count the total number of queries in the input CSV file
  let totalQueries = 0;
  fs.createReadStream(inputCsvFile)
    .pipe(csvParser())
    .on('data', () => {
      totalQueries++;
    })
    .on('end', () => {
      console.log(`Total SPARQL queries to process: ${totalQueries}`);
    });

  const workerFunction = async (row) => {
    const sparqlQuery = row['query']; // Adjust the column name as per your input CSV format
    const parsedQuery = await executeQuery(sparqlQuery);

    // Add the results to the csvData array
    csvData.push({ Query: sparqlQuery, Parsed_Query: parsedQuery });

    // Calculate the completion percentage and show the progress
    const completionPercentage = (csvData.length / totalQueries) * 100;
    console.log(`Progress: ${completionPercentage.toFixed(2)}%`);

    return { Query: sparqlQuery, Parsed_Query: parsedQuery };
  };

  // Use pMap to parallelize the execution of queries
  const workerCount = 4; // Adjust the number of worker threads as per your CPU cores
  await pMap(
    fs.createReadStream(inputCsvFile).pipe(csvParser()),
    workerFunction,
    { concurrency: workerCount }
  );

  // Write the results to output CSV file
  const csv = unparse(csvData, { header: true, delimiter: ',' });
  fs.writeFileSync(outputCsvFile, csv);

  console.log('All SPARQL queries executed and results written to output CSV file.');
}

if (isMainThread) {
  main();
} else {
  parentPort.once('message', (message) => {
    parentPort.postMessage(workerFunction(message));
  });
}
